In [12]:
import requests
import json
import pandas as pd
import itertools
from bs4 import BeautifulSoup
import urllib.parse

In [2]:
config = pd.read_csv('google.config')

In [3]:
# Example usage
api_key = config.key[0]
cx_key = config.cx[0]

In [4]:
def search_companies(query, api_key):
    
    # API endpoint
    url = 'https://places.googleapis.com/v1/places:searchText'
    
    # Data payload
    payload = {
      "textQuery": query
    }
    
    # Headers
    headers = {
      'Content-Type': 'application/json',
      'X-Goog-Api-Key': api_key,
      'X-Goog-FieldMask': 'places.displayName,places.formattedAddress,places.websiteUri'
    }
    
    # Make the POST request
    response = requests.post(url, data=json.dumps(payload), headers=headers)
    
    return response

In [13]:
types = ['Technology', 
         'Tech', 
         'AI', 
         'Artificial Intelligence',
         'Machine Learning',
         'Data Science',
         'Data Analytics',
         # 'Green',
         # 'Climate Change',
         # 'Global Warming',
         'Venture Backed',
         'Seed Funded',
         'Series A Funded',
         'Series B Funded',
         'Series C Funded',
         'Series D Funded'
        ]

where = ['Philadelphia, PA',
         'Greater Philadelphia Region',
         'Havertown, PA',
         'Mainline PA']

queries = [f'{type} Companies in {loc}' for type, loc in itertools.product(types, where)]
queries

['Technology Companies in Philadelphia, PA',
 'Technology Companies in Greater Philadelphia Region',
 'Technology Companies in Havertown, PA',
 'Technology Companies in Mainline PA',
 'Tech Companies in Philadelphia, PA',
 'Tech Companies in Greater Philadelphia Region',
 'Tech Companies in Havertown, PA',
 'Tech Companies in Mainline PA',
 'AI Companies in Philadelphia, PA',
 'AI Companies in Greater Philadelphia Region',
 'AI Companies in Havertown, PA',
 'AI Companies in Mainline PA',
 'Artificial Intelligence Companies in Philadelphia, PA',
 'Artificial Intelligence Companies in Greater Philadelphia Region',
 'Artificial Intelligence Companies in Havertown, PA',
 'Artificial Intelligence Companies in Mainline PA',
 'Machine Learning Companies in Philadelphia, PA',
 'Machine Learning Companies in Greater Philadelphia Region',
 'Machine Learning Companies in Havertown, PA',
 'Machine Learning Companies in Mainline PA',
 'Data Science Companies in Philadelphia, PA',
 'Data Science Com

In [14]:
all_dat = pd.DataFrame()

for query in queries:
    response = search_companies(query, api_key)

    df = pd.DataFrame(response.json().get('places', []))
    if df.shape[0] == 0:
        continue
    df['comp_name'] = df['displayName'].apply(lambda x: x['text'])
    df.drop('displayName', axis = 1, inplace = True)

    all_dat = pd.concat([all_dat, df], axis = 0, ignore_index = True)

In [15]:
all_dat.drop_duplicates(inplace = True)
all_dat

,formattedAddress,websiteUri,comp_name
0,"121 S Broad St floor 9, Philadelphia, PA 19107...",http://getguru.com/,"Guru Technologies, Inc."
1,"1515 Market St Suite 1200, Philadelphia, PA 19...",http://www.pumextech.com/,Pumex Technologies
2,"2400 Market St #200, Philadelphia, PA 19103, USA",http://arcweb.co/,Arcweb Technologies
3,"701 S 50th St Floor 2, Philadelphia, PA 19143,...",http://oceanringtech.com/contact/,Ocean Ring Technologies
4,"601 Walnut St 12th floor, Philadelphia, PA 191...",http://360.technology/,360 Technology LLC
...,...,...,...
708,"435 Devon Park Dr Building 800, Wayne, PA 1908...",http://www.semcap.com/,SemCap
711,"1100 E Hector St #210, Conshohocken, PA 19428,...",https://www.thrivestlink.com/,Thrivest Link Legal Funding™
720,"150 Monument Rd, Bala Cynwyd, PA 19004, USA",http://www.taaffeitecm.com/,Taaffeite Capital Management - TCM Global Inde...
762,"55 S Valley Rd Apt F3, Paoli, PA 19301, USA",https://bigfundings.com/,Big Fundings - Short term business loans


In [16]:
def find_linkedin_url(company_name):
    # Encode the company name for the URL
    query = urllib.parse.quote_plus(f"{company_name} LinkedIn")
    url = f"https://www.google.com/search?q={query}"
    
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"}
    
    response = requests.get(url, headers=headers)
    
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Attempt to find the first LinkedIn URL in the search results
    for a in soup.find_all('a', href=True):
        link = a['href']
        if "linkedin.com/company" in link:
            # Extract the URL from Google's redirect URL if necessary
            start = link.find("http")  # Find the start of the actual URL
            if start != -1:
                end = link.find("&", start)  # Find the end of the actual URL
                if end != -1:
                    return link[start:end]
                return link[start:]
    
    return "LinkedIn URL not found"

In [17]:
li_prof = [find_linkedin_url(company_name) for company_name in list(all_dat['comp_name'])]

In [18]:
li_prof

['https://www.linkedin.com/company/guru-technologies',
 'https://www.linkedin.com/company/pumex-computing-llc',
 'https://www.linkedin.com/company/arcweb',
 'https://www.linkedin.com/company/ocean-ring-technologies',
 'https://www.linkedin.com/company/360technology-usa',
 'https://www.linkedin.com/company/cdw-sirius',
 'https://www.linkedin.com/company/poweredbybosstech',
 'https://www.linkedin.com/company/ibusiness-technologies',
 'https://www.linkedin.com/company/taltech-inc',
 'https://www.linkedin.com/company/tammaninc',
 'https://www.linkedin.com/company/officialgates',
 'https://www.linkedin.com/company/untold-hq',
 'https://www.linkedin.com/company/chirp-technologies-net',
 'https://www.linkedin.com/company/jtm',
 'https://www.linkedin.com/company/kashyak',
 'https://www.linkedin.com/company/paratech-inc.',
 'https://www.linkedin.com/company/netbridge',
 'https://www.linkedin.com/company/trouvaille-consultants',
 'https://www.linkedin.com/company/envative',
 'https://www.linkedi

In [19]:
all_dat['linkedin_profile'] = li_prof

In [20]:
all_dat

,formattedAddress,websiteUri,comp_name,linkedin_profile
0,"121 S Broad St floor 9, Philadelphia, PA 19107...",http://getguru.com/,"Guru Technologies, Inc.",https://www.linkedin.com/company/guru-technolo...
1,"1515 Market St Suite 1200, Philadelphia, PA 19...",http://www.pumextech.com/,Pumex Technologies,https://www.linkedin.com/company/pumex-computi...
2,"2400 Market St #200, Philadelphia, PA 19103, USA",http://arcweb.co/,Arcweb Technologies,https://www.linkedin.com/company/arcweb
3,"701 S 50th St Floor 2, Philadelphia, PA 19143,...",http://oceanringtech.com/contact/,Ocean Ring Technologies,https://www.linkedin.com/company/ocean-ring-te...
4,"601 Walnut St 12th floor, Philadelphia, PA 191...",http://360.technology/,360 Technology LLC,https://www.linkedin.com/company/360technology...
...,...,...,...,...
708,"435 Devon Park Dr Building 800, Wayne, PA 1908...",http://www.semcap.com/,SemCap,https://www.linkedin.com/company/seminalcapital
711,"1100 E Hector St #210, Conshohocken, PA 19428,...",https://www.thrivestlink.com/,Thrivest Link Legal Funding™,https://www.linkedin.com/company/thrivestlink
720,"150 Monument Rd, Bala Cynwyd, PA 19004, USA",http://www.taaffeitecm.com/,Taaffeite Capital Management - TCM Global Inde...,https://www.linkedin.com/company/taaffeite-cap...
762,"55 S Valley Rd Apt F3, Paoli, PA 19301, USA",https://bigfundings.com/,Big Fundings - Short term business loans,https://www.linkedin.com/company/bigfundings


In [21]:
# Function to search for jobs for a given company
def search_jobs(company_name, api_key, cx_key):
    query = f"{company_name} jobs"
    url = f"https://www.googleapis.com/customsearch/v1?key={api_key}&cx={cx_key}&q={query}"
    
    response = requests.get(url)
    result = response.json()

    jobs = []
    for item in result.get('items', []):
        title = item.get('title')
        link = item.get('link')
        jobs.append((title, link))
    
    return jobs

In [23]:
# List of companies to search for
companies = ['Guru Technologies, Inc.', 'Pumex Technologies']  # Add your companies here

# Perform search and display results
for company in companies:
    print(f"Searching for jobs at {company}...")
    jobs = search_jobs(company, api_key, cx_key)
    if jobs:
        print(f"Found jobs for {company}:")
        for title, link in jobs:
            print(f"- {title}: {link}")
    else:
        print("No jobs found.")
    print("\n")  # Add a newline for readability

Searching for jobs at Guru Technologies, Inc....
Found jobs for Guru Technologies, Inc.:
- Guru Technologies | LinkedIn: https://www.linkedin.com/company/guru-technologies
- Guru | LinkedIn: https://www.linkedin.com/company/getguru
- Tech Guru | LinkedIn: https://www.linkedin.com/company/tech-guru
- A Cloud Guru | A Pluralsight Company | LinkedIn: https://www.linkedin.com/company/a-cloud-guru
- Content Guru | LinkedIn: https://www.linkedin.com/company/content-guru
- Content Guru: Jobs | LinkedIn: https://www.linkedin.com/company/content-guru/jobs
- Techguru Inc | LinkedIn: https://www.linkedin.com/company/techguru-inc
- GuRu Wireless | LinkedIn: https://www.linkedin.com/company/guruwirelessinc
- GuruSchools LLC | LinkedIn: https://www.linkedin.com/company/guruschools
- Guru.com | LinkedIn: https://www.linkedin.com/company/gurufreelancing


Searching for jobs at Pumex Technologies...
Found jobs for Pumex Technologies:
- Pumex Computing, LLC | LinkedIn: https://www.linkedin.com/company/p